In [22]:
!pip install lxml


[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [23]:
!pip freeze > ..//requirements.txt

# regressores

In [50]:
import pandas as pd
import numpy as np
from datetime import datetime
df_conflitos = pd.read_csv('../data/raw/conflitos armados-2020-12-09-2023-12-26.csv')
df_conflitos['Date'] = [datetime.fromtimestamp(x).date() for x in df_conflitos.timestamp]
df_conflitos_porpais = df_conflitos\
    .groupby(['Date','event_type','country'])\
    .agg({
          'fatalities':'sum'
        , 'event_id_cnty':'nunique'
    }).reset_index()
df_conflitos_porpais.to_csv('../data/raw/df_conflitos_porpais.csv')
df_conflitos_mundiais = df_conflitos\
    .groupby(['Date','event_type'])\
    .agg({
          'fatalities':'sum'
        , 'event_id_cnty':'nunique'
    }).reset_index()
df_conflitos_mundiais.to_csv('../data/raw/df_conflitos_mundiais.csv')

In [2]:
df_conflitos[['event_type', 'sub_event_type']].drop_duplicates()

,event_type,sub_event_type
0,Battles,Armed clash
1,Violence against civilians,Attack
10,Explosions/Remote violence,Remote explosive/landmine/IED
11,Explosions/Remote violence,Air/drone strike
13,Explosions/Remote violence,Shelling/artillery/missile attack
280,Violence against civilians,Abduction/forced disappearance
306,Battles,Non-state actor overtakes territory
376,Violence against civilians,Sexual violence
453,Explosions/Remote violence,Grenade
464,Battles,Government regains territory


In [51]:
df_conflitos[['event_type', 'sub_event_type']].drop_duplicates().to_csv('../data/raw/tipo_subtipo_eventos.csv')

## conflitos e petroleo

In [43]:
df_conflitos_mundiais = pd.read_csv('../data/raw/df_conflitos_mundiais.csv')
df_conflitos_mundiais['Date'] = pd.to_datetime(df_conflitos_mundiais['Date'])
df_conflitos_mundiais = df_conflitos_mundiais\
.groupby(['Date','event_type'])\
.agg({
        'fatalities':'sum'
     , 'event_id_cnty':'sum'
}).reset_index()
df_conflitos_mundiais = df_conflitos_mundiais.pivot(index='Date', columns='event_type').reset_index().replace(0, None)
df_conflitos_mundiais
# df_conflitos_mundiais.columns = ['Date','Fatalities in Battles','Fatalities in Explosions/Remote Violence','Fatalities in Violence against civillians', 'Qtt Battles','Qtt Explosions/Remote violence','Qtt Violence against civilians']

In [44]:
df = pd.read_html('http://www.ipeadata.gov.br/ExibeSerie.aspx?module=m&serid=1650971490&oper=view', decimal=',', thousands='.', parse_dates=True)[2][1:]
df.columns=['Date','Preco']
df.Date = pd.to_datetime(df.Date, dayfirst=True)
df = df.loc[df.Date.isin(df_conflitos_mundiais.Date.to_list())]
#------------
df_conflitos_mundiais = df_conflitos_mundiais[list(df_conflitos_mundiais['Date'] <= pd.to_datetime('2023-12-01'))]
df_conflitos_mundiais = df_conflitos_mundiais.merge(df, how='inner', on=['Date'])

for col in df_conflitos_mundiais.columns:
    if (col != 'Date'):
        df_conflitos_mundiais[col] = df_conflitos_mundiais[col].astype(float)
        df_conflitos_mundiais[f'log_{col}'] = np.log(df_conflitos_mundiais[col])


from sklearn.preprocessing import MinMaxScaler

cols_to_reescale = ['Fatalities in Battles','Fatalities in Explosions/Remote Violence','Fatalities in Violence against civillians', 'Qtt Battles','Qtt Explosions/Remote violence','Qtt Violence against civilians', 'Preco']
scaler = MinMaxScaler()
df_conflitos_mundiais[["minmax_"+x for x in cols_to_reescale]] = scaler.fit_transform(df_conflitos_mundiais[cols_to_reescale])

In [45]:
import plotly.express as px
import plotly.graph_objs as go

fig = go.Figure()

for col in cols_to_reescale:
    fig.add_trace(go.Scatter(
        x=df_conflitos_mundiais['Date'], y=df_conflitos_mundiais["minmax_"+col],
        mode='lines', yaxis='y', name=col.replace('minmax_','')
        #, custom_data=['year', config.DICT_Y[stat][0], 'country_code']
        )
    )

# hide axes
fig.update_layout(
    hovermode='x unified',
)

fig

In [46]:
import plotly.express as px
import plotly.graph_objs as go

fig = go.Figure()

for col in df_conflitos_mundiais.columns:
    if (col.startswith('log')):
        fig.add_trace(go.Scatter(
            x=df_conflitos_mundiais['Date'], y=df_conflitos_mundiais[col],
            mode='lines', yaxis='y', name=col.replace('log_','')
            #, custom_data=['year', config.DICT_Y[stat][0], 'country_code']
            )
        )

# hide axes
fig.update_layout(
    hovermode='x unified',
)

fig

In [49]:
df_conflitos_mundiais['dt_year_month'] = [x.replace(day=1) for x in df_conflitos_mundiais.Date]
df_grouped_conf_mund = df_conflitos_mundiais.groupby('dt_year_month').agg({
    'Fatalities in Battles':'sum'
    ,'Fatalities in Explosions/Remote Violence':'sum'
    ,'Fatalities in Violence against civillians':'sum'
    ,'Qtt Battles':'sum'
    ,'Qtt Explosions/Remote violence':'sum'
    ,'Qtt Violence against civilians':'sum'
    ,'Preco':'mean'
}).reset_index()
df_grouped_conf_mund['Total Fatalities'] = df_grouped_conf_mund['Fatalities in Battles'] + df_grouped_conf_mund['Fatalities in Explosions/Remote Violence'] + df_grouped_conf_mund['Fatalities in Violence against civillians']
df_grouped_conf_mund['Total Qtt'] = df_grouped_conf_mund['Qtt Battles'] + df_grouped_conf_mund['Qtt Explosions/Remote violence'] + df_grouped_conf_mund['Qtt Violence against civilians']
cols_to_reescale = ['Fatalities in Battles','Fatalities in Explosions/Remote Violence','Fatalities in Violence against civillians', 'Qtt Battles','Qtt Explosions/Remote violence','Qtt Violence against civilians', 'Preco','Total Qtt','Total Fatalities']
scaler = MinMaxScaler()
df_grouped_conf_mund[["minmax_"+x for x in cols_to_reescale]] = scaler.fit_transform(df_grouped_conf_mund[cols_to_reescale])

fig = go.Figure()

for col in cols_to_reescale:
    fig.add_trace(go.Scatter(
        x=df_grouped_conf_mund['dt_year_month'], y=df_grouped_conf_mund["minmax_"+col],
        mode='lines', yaxis='y', name=col.replace('minmax_','')
        #, custom_data=['year', config.DICT_Y[stat][0], 'country_code']
        )
    )

# hide axes
fig.update_layout(
    hovermode='x unified',
)

fig